# 如何派发自定义回调事件
:::info 前提条件
本指南假定您已熟悉以下概念：
- [回调函数](/docs/concepts/callbacks)- [自定义回调处理程序](/docs/how_to/custom_callbacks)- [Astream Events API](/docs/concepts/streaming/#astream_events) `astream_events` 方法将呈现自定义回调事件。:::
在某些情况下，您可能希望从 [Runnable](/docs/concepts/runnables) 内部派发一个自定义回调事件，以便该事件能够被捕获并处理。在自定义回调处理程序中或通过 [Astream 事件 API](/docs/concepts/streaming/#astream_events) 实现。
例如，如果你有一个包含多个步骤的长时间运行工具，你可以在步骤之间分派自定义事件，并利用这些自定义事件来监控进度。你也可以将这些自定义事件展示给应用程序的最终用户，让他们了解当前任务的进展情况。
要派发一个自定义事件，你需要确定该事件的两个属性：`name`（名称）和 `data`（数据）。
| 属性 | 类型 | 描述                                                                                              ||-----------|------|----------------------------------------------------------------------------------------------------------|| name      | str  | 用户为事件定义的名称。                                                                       || data      | 任意类型  | 与事件关联的数据。可以是任何内容，但我们建议使其可JSON序列化。 |

:::重要* 调度自定义回调事件需要 `langchain-core>=0.2.15`。* 自定义回调事件只能从现有的 `Runnable` 内部触发。* 如果使用 `astream_events`，必须指定 `version='v2'` 才能看到自定义事件。* LangSmith 目前不支持发送或渲染自定义回调事件。:::

:::caution 兼容性LangChain 无法自动将配置（包括 `astream_events()` 所需的回调）传播到子可运行对象，如果您在 Python<=3.10 中运行异步代码。这是您可能无法从自定义可运行对象或工具中看到事件发出的常见原因。
如果你正在运行 Python<=3.10 版本，在异步环境中需要手动将 `RunnableConfig` 对象传递给子可运行对象。关于如何手动传递配置的示例，请参阅下方 `bar` RunnableLambda 的实现。
如果您正在运行 Python>=3.11 版本，在异步环境中 `RunnableConfig` 会自动传播到子可运行对象。不过，如果您的代码可能在其他 Python 版本中运行，手动传播 `RunnableConfig` 仍然是一个好习惯。:::

In [ ]:
# | output: false
# | echo: false

%pip install -qU langchain-core

## Astream 事件API
使用自定义事件最有用的方式是通过 [Astream 事件 API](/docs/concepts/streaming/#astream_events)。
我们可以使用 `async` `adispatch_custom_event` API 在异步环境中派发自定义事件。

:::重要
要通过 astream events API 查看自定义事件，您需要使用较新的 `v2` 版 `astream_events` API。好的,我会按照要求将英文翻译成中文,并保持markdown格式一致。以下是一个示例翻译:

# 欢迎使用翻译助手

这是一个标准的markdown格式翻译示例。

## 二级标题

- 列表项1
- 列表项2
- 列表项3

**加粗文本** 和 *斜体文本*

[链接文本](https://example.com)

> 引用内容

```
代码块
```

表格示例:

| 列1 | 列2 |
|-----|-----|
| 数据1 | 数据2 |

我会确保:
1. 保持原有markdown格式
2. 只输出翻译后的内容
3. 不包含额外的标记符号

请提供需要翻译的英文内容。

In [3]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


@RunnableLambda
async def foo(x: str) -> str:
    await adispatch_custom_event("event1", {"x": x})
    await adispatch_custom_event("event2", 5)
    return x


async for event in foo.astream_events("hello world", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'hello world'}, 'name': 'foo', 'tags': [], 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'event1', 'tags': [], 'metadata': {}, 'data': {'x': 'hello world'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'event2', 'tags': [], 'metadata': {}, 'data': 5, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'foo', 'tags': [], 'metadata': {}, 'data': {'chunk': 'hello world'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'hello world'}, 'run_id': 'f354ffe8-4c22-4881-890a-c1cad038a9a6', 'name': 'foo', 'tags': [], 'metadata': {}, 'parent_ids': []}


在 Python 3.10 及更早版本中，您必须手动传递配置！

In [7]:
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


@RunnableLambda
async def bar(x: str, config: RunnableConfig) -> str:
    """An example that shows how to manually propagate config.

    You must do this if you're running python<=3.10.
    """
    await adispatch_custom_event("event1", {"x": x}, config=config)
    await adispatch_custom_event("event2", 5, config=config)
    return x


async for event in bar.astream_events("hello world", version="v2"):
    print(event)

{'event': 'on_chain_start', 'data': {'input': 'hello world'}, 'name': 'bar', 'tags': [], 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'metadata': {}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'event1', 'tags': [], 'metadata': {}, 'data': {'x': 'hello world'}, 'parent_ids': []}
{'event': 'on_custom_event', 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'event2', 'tags': [], 'metadata': {}, 'data': 5, 'parent_ids': []}
{'event': 'on_chain_stream', 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'bar', 'tags': [], 'metadata': {}, 'data': {'chunk': 'hello world'}, 'parent_ids': []}
{'event': 'on_chain_end', 'data': {'output': 'hello world'}, 'run_id': 'c787b09d-698a-41b9-8290-92aaa656f3e7', 'name': 'bar', 'tags': [], 'metadata': {}, 'parent_ids': []}


## 异步回调处理器
您还可以通过异步回调处理程序来消费已分发的事件。

In [8]:
from typing import Any, Dict, List, Optional
from uuid import UUID

from langchain_core.callbacks import AsyncCallbackHandler
from langchain_core.callbacks.manager import (
    adispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


class AsyncCustomCallbackHandler(AsyncCallbackHandler):
    async def on_custom_event(
        self,
        name: str,
        data: Any,
        *,
        run_id: UUID,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print(
            f"Received event {name} with data: {data}, with tags: {tags}, with metadata: {metadata} and run_id: {run_id}"
        )


@RunnableLambda
async def bar(x: str, config: RunnableConfig) -> str:
    """An example that shows how to manually propagate config.

    You must do this if you're running python<=3.10.
    """
    await adispatch_custom_event("event1", {"x": x}, config=config)
    await adispatch_custom_event("event2", 5, config=config)
    return x


async_handler = AsyncCustomCallbackHandler()
await foo.ainvoke(1, {"callbacks": [async_handler], "tags": ["foo", "bar"]})

Received event event1 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: a62b84be-7afd-4829-9947-7165df1f37d9
Received event event2 with data: 5, with tags: ['foo', 'bar'], with metadata: {} and run_id: a62b84be-7afd-4829-9947-7165df1f37d9


1

## 同步回调处理器
让我们看看如何在同步环境中使用 `dispatch_custom_event` 派发自定义事件。
你**必须**在现有的 `Runnable` 中调用 `dispatch_custom_event`。

In [5]:
from typing import Any, Dict, List, Optional
from uuid import UUID

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.callbacks.manager import (
    dispatch_custom_event,
)
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.config import RunnableConfig


class CustomHandler(BaseCallbackHandler):
    def on_custom_event(
        self,
        name: str,
        data: Any,
        *,
        run_id: UUID,
        tags: Optional[List[str]] = None,
        metadata: Optional[Dict[str, Any]] = None,
        **kwargs: Any,
    ) -> None:
        print(
            f"Received event {name} with data: {data}, with tags: {tags}, with metadata: {metadata} and run_id: {run_id}"
        )


@RunnableLambda
def foo(x: int, config: RunnableConfig) -> int:
    dispatch_custom_event("event1", {"x": x})
    dispatch_custom_event("event2", {"x": x})
    return x


handler = CustomHandler()
foo.invoke(1, {"callbacks": [handler], "tags": ["foo", "bar"]})

Received event event1 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: 27b5ce33-dc26-4b34-92dd-08a89cb22268
Received event event2 with data: {'x': 1}, with tags: ['foo', 'bar'], with metadata: {} and run_id: 27b5ce33-dc26-4b34-92dd-08a89cb22268


1

## 后续步骤
您已经了解了如何发送自定义事件，可以查看更深入的指南 [流事件](/docs/how_to/streaming/#using-stream-events)，这是利用自定义事件最简单的方法。